In [13]:
import numpy as np
import casadi as ca
from SE3Lie_ca import se3, SE3, dot_plot_draw, so3_wedge, so3_vee

#http://asrl.utias.utoronto.ca/~tdb/bib/barfoot_ser17.pdf

In [14]:
pt_true = ca.SX([1,2, 3, 0.1, 0.2, 0.3]) #true location [x y z theta1 theta2 theta3]

#Camera from corners in inertial frame
pt_cam = (ca.SX([[1,-2,3,1],[1,2,3,1],[1,2,-3,1],[1,-2,-3,1],[-1,2,3,1],[-1,-2,-3,1],[-1,-2,3,1],[-1,2,-3,1]])).T

def_weight = ca.SX.ones(8,1) #set weight of each points to ones

In [15]:
def find_Tau(T):
    '''
    Accepts SE3 elements operator (T_op) and find big Adjoint SE(3)
    '''
    return SE3.Ad(T)

In [16]:
def ca_avg(x,w=def_weight):
    '''
    Find average value (elements of 4 rows) of the given Matrix in Casadi;
    return (average_value, total_weight)
    '''
    sum = 0
    W = 0
    for i in range(x.shape[1]):
        sum += x[:,i]
        W += w[i] #assume weight on each camera points is 1
    return sum/W ,W



ca_avg(pt_cam)

(SX(@1=0, [@1, @1, @1, 1]), SX(8))

In [17]:
def find_y(pt_cam, pt_true):
    '''
    Accepts camera points and points of body frame and return the camera frame location from body to each camera
    '''
    T_true = se3.exp(se3.wedge(pt_true))
    yj= ca.SX.zeros(pt_cam.shape[1],4)
    w=0
    for i in range(pt_cam.shape[1]):
        yj[i,0:4] = T_true@pt_cam[:,i]
    return yj.T

find_y(pt_cam,pt_true)

SX(@1=1, 
[[3.13266, 2, 0.73885, 1.87151, 0.12849, 1.11022e-16, 1.26115, -1.13266], 
 [0.197678, 4, 4.40819, 0.605865, 3.39413, -2.22045e-16, -0.408188, 3.80232], 
 [5.49066, 6, 0.148258, -0.36108, 6.36108, 0, 5.85174, 0.509338], 
 [@1, @1, @1, @1, @1, @1, @1, @1]])

In [18]:
def find_eps_st(pt_cam,pt_true,Top,w = def_weight):
    ''' 
    Find Barfoot epsilon_star term with weight, w, as array weights
    '''
    C_op = Top[:3,:3]
    r_op = -1 * C_op.T@Top[:3,3]

    P = ca_avg(pt_cam,w)[0]
    W = ca_avg(pt_cam,w)[1]
    y = find_y(pt_cam,pt_true)
    Y = ca_avg(y,w)[0]
    Tau = find_Tau(Top)

    ## Finding I term
    I = ca.SX.zeros(3,3) 
    for i in range(pt_cam.shape[1]):
        I += w[i]* so3_wedge(pt_cam[:,i]-P)@so3_wedge(pt_cam[:,i]-P)
    I = - I/W

    ## Finding M term
    c1 = ca.vertcat( ca.horzcat(ca.SX.eye(3),ca.SX.zeros(3,3)) , ca.horzcat(so3_wedge(P),ca.SX.eye(3)))
    c2 = ca.vertcat( ca.horzcat(ca.SX.eye(3),ca.SX.zeros(3,3)) , ca.horzcat(ca.SX.zeros(3,3),I)  )
    c3 = ca.vertcat( ca.horzcat(ca.SX.eye(3),-so3_wedge(P)), ca.horzcat(ca.SX.zeros(3,3),ca.SX.eye(3)))
    M = c1@c2@c3
    
    # ## Finding a term using summation method
    # A_val = ca.SX.zeros(6,1)
    # for i in range(pt_cam.shape[0]):
    #     zj = Top@(pt_cam[:,i])
    #     # print(ca.horzcat(zj[3]@ca.SX.eye(3),-1*so3_wedge(zj[:3])))
    #     zj_circle = (ca.vertcat( 
    #         ca.horzcat(zj[3]@ca.SX.eye(3),-1*so3_wedge(zj[:3])),
    #         ca.SX.zeros(1,6))) #circle notation from barfoot 4x6 matrix
    #     A_val += w[i] * zj_circle.T @ ((y[:,i])-zj)
    # A_val =  A_val/W
    # print(A_val)

    ## Finding a term using matrix method
    W_term = 0
    for i in range(pt_cam.shape[1]):
        W_term += w[i] * (y[:,i]-Y)@ca.transpose(pt_cam[:,i]-P)
    W_term = W_term/W
    
    ## Finding b term
    b = ca.SX.zeros(1,3)
    for i in range(3):
        b[i] = ca.trace(so3_wedge(ca.SX.eye(3)[i,:])@C_op@W_term[:3,:3].T)

    ## Find a
    t1 = Y[:3]-C_op@(P[:3]-r_op)
    t2 = b.T - so3_wedge(Y[:3])@C_op@(P[:3]-r_op)
    A_val = ca.vertcat(t1, t2)
    
    return Tau@ca.inv(M)@Tau.T@A_val



In [20]:
R_op = ca.SX.eye(3)
t_op = ca.SX.zeros(3,1)

#initialized T_op
T_op = ca.vertcat(ca.horzcat(R_op, -R_op@t_op),ca.horzcat(ca.SX.zeros(1,R_op.shape[1]),1))

T_op

SX(@1=1, @2=0, 
[[@1, 00, 00, @2], 
 [00, @1, 00, @2], 
 [00, 00, @1, @2], 
 [@2, @2, @2, @1]])

In [21]:
iter = 0
eps_st = None
tol = 0.0000001 # Acceptable error
count = 0

while iter < 6:
    eps_st = find_eps_st(pt_cam,pt_true,T_op)
    lieAlg = SE3.log(T_op) #Convert o Lie algebra element using log map
    T_op = se3.exp(se3.wedge(eps_st)) @ T_op
    
    iter = 0
    count += 1
    for i in range(pt_true.shape[0]):
        if ca.fabs(se3.vee(lieAlg)[i] - pt_true[i]) < tol:
            iter += 1
            continue
        else:
            break


In [22]:
''' 
Final Converged Value in [x y z theta1 theta2 theta3] is
'''

se3.vee(lieAlg)

SX([1, 2, 3, 0.1, 0.2, 0.3])

In [23]:
'''
Number of iterations before converges
'''
count

4